In [1]:
import json
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import linprog

# Bài toán 1: phân bổ tiền cho các mã cho trước dựa trên tỉ lệ lãi và rủi ro mã đó đem lại

có $x_1, x_2, x_3$ lần lượt là số tiền đầu tư vào mã 1, 2 và 3

- Hàm mục tiêu là số tiền rủi ro là ít nhất (e_1, e_2, e_3 lần lượt là tỷ lệ rủi ro được tính băng sum(Pred_moc != truth)/number_of_pred) 

$$e_1 * x1 + e_2 * x2 + e3*x3 <- min$$

- Ràng buộc 1: tổng tiền nhỏ hơn hoặc bằng số tiền hiện có 

$$
x_1 + x_2 + x3 \leq TotalBudget
$$

- Ràng buộc 2: tổng tiền lãi phải đạt được một ngưỡng nhất định (>= 50%, có thể lấy số khác 50%) (với p_1, p_2, p_3 lần lượt là tỷ lệ lãi tính trên pred)

$$
p_1 x_1 + p_2 x_2 + p_3 x_3 >= 0,5 * TotalBudget
$$

- Ràng buộc về miền mỗi ma mua ko quá 5-0 phần trăm số vốn: 

$$
x_i \geq 0 \hspace{0.25cm} \text{and}  \hspace{0.25cm} x_i <= TotalBudget * 0.5
$$

In [3]:
with open('result.json') as f:
    result_dat = json.load(f)
print(result_dat.keys())

dict_keys(['ITD', 'DXG', 'OGC', 'ELC', 'BMP', 'BTT', 'IJC', 'BWE', 'FCN', 'KDC', 'DTL', 'KDH', 'VPS', 'VRC', 'KPF', 'DRL', 'IDI', 'DSN', 'OPC', 'C32', 'FDC', 'BTP', 'FIT', 'KHP', 'DRC', 'DVP', 'PAN', 'VPD', 'VSC'])


In [4]:
print(len(result_dat.keys()))

29


In [5]:
# risk calculator from 1d data
type_chose = '1d'
max_date = 326
performance_date = int(max_date / 2)
for ticker in result_dat:
    per_error = result_dat[ticker][type_chose]['middle_open_close']['percentage_error'][:performance_date]
    per_change = result_dat[ticker][type_chose]['middle_open_close']['percent_changes'][:performance_date]
    per_error = np.array(per_error)
    per_change = np.array(per_change)
    
    # pererror 1percent error can make the game difference
    count_outliers = np.where((np.abs(per_error) > 1))[0].shape[0]
    risk_error = count_outliers / performance_date
    print(f'{ticker} error risk: {risk_error}')
    
    # perchange
    q1 = np.percentile(per_change, 25)
    q3 = np.percentile(per_change, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr if q1 - 1.5 * iqr < -7 else -7
    upper_bound = q3 + 1.5 * iqr if q3 + 1.5 * iqr > 7 else 7
    count_outliers = np.where((per_change < lower_bound) | (per_change > upper_bound))[0].shape[0]
    risk_change = count_outliers / performance_date
    result_dat[ticker][type_chose]['middle_open_close']['risk_error'] = risk_error * 0.5 + risk_change * 0.5
    
    

ITD error risk: 0.5460122699386503
DXG error risk: 0.4110429447852761
OGC error risk: 0.5030674846625767
ELC error risk: 0.9631901840490797
BMP error risk: 0.8650306748466258
BTT error risk: 0.656441717791411
IJC error risk: 0.44171779141104295
BWE error risk: 0.4233128834355828
FCN error risk: 0.36809815950920244
KDC error risk: 0.10429447852760736
DTL error risk: 0.3987730061349693
KDH error risk: 0.36809815950920244
VPS error risk: 0.4785276073619632
VRC error risk: 0.5828220858895705
KPF error risk: 0.656441717791411
DRL error risk: 0.7055214723926381
IDI error risk: 0.48466257668711654
DSN error risk: 0.754601226993865
OPC error risk: 0.44785276073619634
C32 error risk: 0.17791411042944785
FDC error risk: 0.6134969325153374
BTP error risk: 0.5950920245398773
FIT error risk: 0.36809815950920244
KHP error risk: 0.3558282208588957
DRC error risk: 0.8404907975460123
DVP error risk: 0.9386503067484663
PAN error risk: 0.3619631901840491
VPD error risk: 0.7116564417177914
VSC error risk:

In [6]:
# assume the player use money in day performance + 1
day_play = int(performance_date / 5) + 1

type_chose = '1w'

# take top all the percent change of this day
# take all risk_error of this day
all_percent_change = []
all_risk_error = []
for ticker in result_dat:
    per_change = result_dat[ticker][type_chose]['middle_open_close']['percent_changes'][day_play]
    risk_error = result_dat[ticker]['1d']['middle_open_close']['risk_error']
    all_percent_change.append(per_change)
    all_risk_error.append(risk_error)

In [ ]:
TOTAL_MON = 200_000_000

# progame 1: maximize the sum of all percent change
obj = -np.array(all_percent_change) / 100

# the total money risk should be less than 50%
lhs_ineq = [[1 for _ in all_percent_change], all_risk_error]
rhs_ineq = [TOTAL_MON, TOTAL_MON * 0.5]

# one ticket can't buy greater than 20%
bnd = [(0, TOTAL_MON * 0.2) for _ in all_percent_change]  

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method="revised simplex")

print(opt)

print('The total money risk:', np.sum(np.array(all_risk_error) * opt.x) / TOTAL_MON)
print('The total percent change:', np.round(-opt.fun / TOTAL_MON * 100), "%")
print('The total money:', np.sum(opt.x))
print('The total ticket:', np.sum(opt.x > 0))

# print as money format
for elm in np.where(opt.x > 0)[0]:
    print("----->", list(result_dat.keys())[elm],  "{:,.0f} VND".format(opt.x[elm]))

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -5963423.411895576
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nit: 5
The total money risk: 0.3239263803680982
The total percent change: 3.0 %
The total money: 200000000.0
The total ticket: 5
-----> BTT 40,000,000 VND
-----> IJC 40,000,000 VND
-----> FDC 40,000,000 VND
-----> BTP 40,000,000 VND
-----> DRC 40,000,000 VND


In [ ]:
TOTAL_MON = 200_000_000

# progame 2: minizie the sum of all risk error
obj = np.array(all_risk_error)

# the total money profit should be greater than 50%
lhs_ineq = [[1 for _ in all_percent_change], -np.array(all_percent_change) / 100]
rhs_ineq = [TOTAL_MON, -1 * TOTAL_MON * 0.5]

# one ticket can't buy greater than 20%
bnd = [(0, TOTAL_MON * 0.35) for _ in all_percent_change]  

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method="revised simplex")

print(opt)

print('The total money risk:', np.sum(np.array(all_risk_error) * opt.x) / TOTAL_MON)
print('The total percent change:', np.sum(np.array(all_percent_change) * opt.x) / TOTAL_MON)
print('The total money:', np.sum(opt.x))
print('The total ticket:', np.sum(opt.x > 0))

# print as money format
for elm in np.where(opt.x > 0)[0]:
    print("----->", list(result_dat.keys())[elm],  "{:,.0f} VND".format(opt.x[elm]))


 message: The problem appears infeasible, as the phase one auxiliary problem terminated successfully with a residual of 9.3e+07, greater than the tolerance 1e-12 required for the solution to be considered feasible. Consider increasing the tolerance to be greater than 9.3e+07. If this tolerance is unacceptably large, the problem is likely infeasible.
 success: False
  status: 2
     fun: 58251533.74233128
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nit: 3
The total money risk: 0.2912576687116564
The total percent change: 3.584948446969233
The total money: 200000000.0
The total ticket: 3
-----> BTT 70,000,000 VND
-----> IJC 60,000,000 VND
-----> BTP 70,000,000 VND
